In [14]:
import pyarrow.parquet as pq
import pyarrow as pa
import pandas as pd
import datetime as dt
import numpy as np

In [15]:
# The predicted period x days
period = 3

In [16]:
air_visit_df = pd.read_csv('input/air_visit_data.csv')
sample_submission_df = pd.read_csv('input/sample_submission.csv')

In [17]:
# Debug
sample_submission_df = sample_submission_df[:100]

In [18]:
feature_df = sample_submission_df.copy()
feature_df['feature1'] = pd.Series(np.random.randn(len(sample_submission_df)), index=sample_submission_df.index)
feature_df['feature2'] = pd.Series(np.random.randn(len(sample_submission_df)), index=sample_submission_df.index)

In [19]:
def predict(*x):
    return np.random.randn()

In [20]:
def prev_date_str(sdate, dlt=1):
    return  dt.datetime.strftime(dt.datetime.strptime(sdate, '%Y-%m-%d') - dt.timedelta(dlt), '%Y-%m-%d')

def ext_air_store_id(asidd):
    return '_'.join(asidd.split('_')[:2])

def ext_date(asidd):
    return asidd.split('_')[-1]

def air_store_id_index(df):
    asid_df = df['id'].apply(lambda x: ext_air_store_id(x)).to_frame()
    tdf = asid_df.groupby('id').count()
    return tdf.index

def predicted_date(df):
    visit_date = df['id'].apply(lambda x: ext_date(x)).to_frame()
    return (visit_date.min().get(0), visit_date.max().get(0), visit_date['id'])

In [21]:
first_date, last_date, predicted_dates = predicted_date(feature_df)

In [22]:
base_df = pd.DataFrame(air_store_id_index(feature_df))
base_df.columns = ['air_store_id']

In [23]:
for dlt in range(period, 0, -1):
    target_date = prev_date_str(first_date, dlt)
    base_df = pd.merge(base_df, air_visit_df[air_visit_df['visit_date'] ==  target_date][['air_store_id', 'visitors']], how='left', on='air_store_id')
    base_df.columns = pd.Index(np.append(base_df.columns[:-1].values, target_date))

In [24]:
for d in predicted_dates.tolist():
    base_df[d] = pd.Series(np.nan, index=base_df.index)

In [25]:
for idx, row in feature_df.iterrows():
    asid = ext_air_store_id(row['id'])
    date = ext_date(row['id'])
    tmp_df = pd.concat([row.to_frame().transpose(), base_df[base_df['air_store_id'] == asid][[prev_date_str(date, i) for i in  range(period, 0, -1)]]], axis=1)
    predicted = predict(tmp_df.drop('id', axis=1).drop('visitors', axis=1).as_matrix())
    base_df.loc[base_df['air_store_id'] == asid, [date]] = predicted

In [26]:
base_df

,air_store_id,2017-04-20,2017-04-21,2017-04-22,2017-04-23,2017-04-24,2017-04-25,2017-04-26,2017-04-27,2017-04-28,...,2017-05-22,2017-05-23,2017-05-24,2017-05-25,2017-05-26,2017-05-27,2017-05-28,2017-05-29,2017-05-30,2017-05-31
0,air_00a91d42b08b08d9,38.0,55,18,-0.501261,0.588501,-0.669021,-1.769047,-0.950176,0.784808,...,0.627258,-0.236490,0.145755,-0.743006,0.354591,0.969835,-0.942242,0.146236,1.822236,-0.297505
1,air_0164b9927d20bcc3,1.0,26,6,-0.599731,-1.743496,-0.436793,0.341204,-1.639602,1.270212,...,-0.205896,0.474736,0.069402,0.454990,-0.070340,-0.284013,-1.421816,-0.879715,0.642331,-1.554984
2,air_0241aa3964b7f861,NaN,3,13,-0.453693,-0.860413,-0.462592,0.109374,-0.101202,0.225073,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
